In [ ]:
import networkx as nx
import nxviz as nv
import pandas as pd
import matplotlib as plt
import community

In [ ]:
df_account = pd.read_csv('All_Fake_without_debunkers_Updated.csv')

In [ ]:
df_cluster = pd.read_csv('Cluster_Tweet_Final.csv')

In [ ]:
df_account = pd.read_csv('All_Fake_without_debunkers_Updated.csv')

In [ ]:
df_cluster = pd.read_csv('Cluster_Tweet_Final.csv')
df_cluster = df_cluster.dropna(subset=['Cluster_ID'])
df_cluster = df_cluster.dropna(subset = 'User_ID')
df_cluster.User_ID = df_cluster.User_ID.astype(int)
df_cluster = df_cluster.dropna(subset=['Cluster_ID'])

In [ ]:
df_cluster[['Cluster_ID', 'Tweet_ID', 'User_ID']].dtypes

In [ ]:
df_cluster2 = pd.merge(df_cluster[['Cluster_ID', 'Tweet_ID', 'User_ID']], df_account[['User_ID', 'party', 'Political']], how = 'left')

In [ ]:
df_user_list = df_cluster2[df_cluster2.isna().any(axis=1)].groupby('User_ID').count().reset_index().User_ID.tolist()

In [ ]:
df_cluster = df_cluster.rename(columns = {'User_ScreenName':'ScreenName'})

In [ ]:
df_cluster.ScreenName = '@' + df_cluster.ScreenName

In [ ]:
df_cluster.ScreenName

In [ ]:
screen_name_list = list(set(df_cluster[df_cluster.User_ID.isin(df_user_list)].ScreenName.tolist()))

In [ ]:
df_account.loc[df_account.ScreenName == screen_name_list[7]].User_ID.tolist()[0]

In [ ]:
df_cluster.groupby(['ScreenName', 'User_ID']).count().reset_index()[['ScreenName', 'User_ID']]

In [ ]:
df_cluster.loc[df_cluster.ScreenName == duplicate_screen_names.reset_index().ScreenName.tolist()[i], 'User_ID'] = 1317831320739827713

In [ ]:
for i in range(len(duplicate_screen_names.reset_index().ScreenName.tolist())):
    sc = duplicate_screen_names.reset_index().ScreenName.tolist()[i]
    uid_list = list(set(df_cluster.loc[df_cluster.ScreenName == duplicate_screen_names.reset_index().ScreenName.tolist()[i]].User_ID))
    for j in uid_list:
        if len(df_account.loc[df_account.User_ID == j]) == 0:
            continue
        else:
            df_cluster.loc[df_cluster.ScreenName == duplicate_screen_names.reset_index().ScreenName.tolist()[i], 'User_ID'] = j
    

In [ ]:
df_cluster.to_csv('Cluster_Tweet_Final.csv', index = False)

In [ ]:
df_cluster2 = pd.merge(df_cluster[['Cluster_ID', 'Tweet_ID', 'User_ID']], df_account[['User_ID', 'party', 'Political']], how = 'left')

In [ ]:
import networkx as nx
from itertools import combinations
import pandas as pd


G = nx.Graph()

user_clusters = df_cluster2.groupby('User_ID')['Cluster_ID'].unique().to_dict()

for pair in combinations(user_clusters.keys(), 2):
    user1, user2 = pair
    common_clusters = set(user_clusters[user1]).intersection(user_clusters[user2])
    if len(common_clusters) >= 5:
        weight = len(common_clusters)
        G.add_edge(user1, user2, weight=weight)

print("Number of nodes:", G.number_of_nodes())
print("Number of edges:", G.number_of_edges())

for u, v, data in G.edges(data=True):
    print(f"Edge ({u}, {v}): Weight = {data['weight']}")

In [ ]:
partition = community.best_partition(G, resolution = 1)

community_data = []

for community_id in set(partition.values()):
    nodes_in_community = [node for node, comm_id in partition.items() if comm_id == community_id]
    
    subgraph = G.subgraph(nodes_in_community)
    
    num_nodes = subgraph.number_of_nodes()
    num_edges = subgraph.number_of_edges()
    density = nx.density(subgraph)
    avg_degree = sum(dict(subgraph.degree()).values()) / num_nodes
    clustering_coefficient = nx.average_clustering(subgraph)
    partition2 = community.best_partition(subgraph)
    modularity = community.modularity(partition2, subgraph)
    edge_weights = [data['weight'] for _, _, data in subgraph.edges(data=True)]
    average_weight = sum(edge_weights) / len(edge_weights) if edge_weights else 0
    
    community_data.append((community_id, num_nodes, num_edges, density, avg_degree, clustering_coefficient, modularity, average_weight))

df_community = pd.DataFrame(community_data, columns=['Community_ID', 'Num_Nodes', 'Num_Edges', 'Density', 'Avg_Degree', 'Clustering_Coefficient', 'Modularity', 'Avg_Weight'])


In [ ]:
df_community = df_community.sort_values('Num_Nodes', ascending = False).reset_index(drop= True)

In [ ]:
df_community['Community_ID'] = 'Wild_Community_' + df_community['Community_ID'].astype('str')

In [ ]:
df_community.sum()

In [ ]:
len(df_community.sort_values('Num_Nodes', ascending = False))

In [ ]:
community.modularity(partition, G)

In [ ]:
df_community.to_csv('Wild_Community.csv', index = False)

In [ ]:
df_community

In [ ]:
communities[31]

In [ ]:
communities = {}
for node, community_id in partition.items():
    if community_id not in communities:
        communities[community_id] = []
    communities[community_id].append(node)

for community_id, nodes in communities.items():
    print(f"Community {community_id}: {len(nodes)}")

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import community


target_community = 31

nodes_in_community = [node for node, comm_id in partition.items() if comm_id == target_community]

community_subgraph = G.subgraph(nodes_in_community)

pos = nx.spring_layout(community_subgraph)  # Or any other layout you prefer
nx.draw(community_subgraph, pos, with_labels=True, node_color='skyblue', node_size=100, font_size=12)
plt.title('Community ' + str(target_community))
plt.show()

In [ ]:
community_subgraph 

In [ ]:
pd.DataFrame(partition, columns=['Community_ID'])

In [ ]:
df_partition

In [ ]:
df_partition = pd.DataFrame(partition.items(), columns=['User_ID', 'Community_ID'])

In [ ]:
df_partition['Community_ID'] = 'Community_' + df_partition['Community_ID'].astype('str')

In [ ]:
df_partition.groupby('Community_ID').count()

In [ ]:
nx.write_gpickle(G, 'Community_Graph_Wild_Campaign.gpickle')

In [ ]:
import json

file_path = "Community_Partition_List_Wild_Campaign.json"

with open(file_path, 'w') as f:
    for key, value in partition.items():
        f.write(f"{key}: {value}\n")

print("Dictionary has been saved successfully.")

In [ ]:
df_partition.to_csv('User_Community_Map.csv', index = False)

In [ ]:
len(df_partition.groupby('Community_ID').count().reset_index().sort_values('User_ID', ascending = False))

In [ ]:
df_partition.groupby('Community_ID').count().reset_index().sort_values('User_ID', ascending = False).User_ID.tolist()

# Alt Campaign Community

In [ ]:
df_alt_influence = pd.read_csv('AltNews Report Check.csv')
df_alt_small = df_alt_influence.loc[df_alt_influence.Campaign == 'Influence'][['News_Number', 'Fact_Check_Link']]

df_alt_all = pd.read_csv('Collected_Fake_Tweets_Still_Available.csv')
df_account = pd.read_csv('All_Fake_without_debunkers_Updated.csv')

df_alt_all_influence = pd.merge(df_alt_all, df_alt_small, on = 'Fact_Check_Link').drop(columns = 'Unnamed: 0')
df_merge_info = pd.merge(df_alt_all_influence[['User_ID', 'News_Number']], df_account[['User_ID', 'Political', 'party', 'ScreenName']])

In [ ]:
alt_number= df_alt_all_influence.groupby('News_Number').count().reset_index()[['News_Number', 'Tweet_ID']]

In [ ]:
alt_number_5 = alt_number.loc[alt_number.Tweet_ID >= 5].rename(columns = {'Tweet_ID': 'Campaign_Size'})

In [ ]:
alt_number_5['Campaign_ID'] = 'Alt_'  + alt_number_5['News_Number'].astype('str')

In [ ]:
df_alt_all_influence[df_alt_all_influence.News_Number.isin(alt_number_5.News_Number.tolist())][['User_ID', 'News_Number']]

In [ ]:
alt_camp = alt_number_5[['Campaign_ID','Campaign_Size' ]]

# Wild Campaign Community

In [ ]:
df_cluster = pd.read_csv('Cluster_Tweet_Final.csv')

In [ ]:
df_cluster.Cluster_ID = df_cluster.Cluster_ID.astype('int')

In [ ]:
df_cl_n = df_cluster.groupby('Cluster_ID').count().reset_index(drop= False)[['Cluster_ID', 'Tweet_ID']]

In [ ]:
df_cluster5 = df_cl_n.loc[df_cl_n.Tweet_ID >= 5 ].rename(columns = { 'Tweet_ID': 'Campaign_Size'})

In [ ]:
df_cluster5['Campaign_ID'] = 'Wild_'  + df_cluster5['Cluster_ID'].astype('str')

In [ ]:
wild_camp = df_cluster5[['Campaign_ID', 'Campaign_Size']]

In [ ]:
df_cluster[df_cluster.Cluster_ID.isin(df_cluster5.Cluster_ID.tolist())][['Cluster_ID', 'User_ID']]

In [ ]:
camp_all = pd.concat([alt_camp, wild_camp]).reset_index(drop = True).sort_values('Campaign_Size', ascending = False).iloc[1:].reset_index(drop = True)

In [ ]:
camp_all 

In [ ]:
alt_number_5

In [ ]:
df_acc_alt = df_alt_all_influence[df_alt_all_influence.News_Number.isin(alt_number_5.News_Number.tolist())][['User_ID', 'News_Number']]



merged_df = pd.merge(df_acc_alt, df_partition, on='User_ID')

pivot_df = merged_df.pivot_table(index='News_Number', columns='Community_ID', aggfunc='size', fill_value=0)

pivot_df.reset_index(inplace=True)

pivot_df.columns.name = None  # Remove the 'Community_ID' label
pivot_df.columns = [f"{col}" if col != 'News_Number' else col for col in pivot_df.columns]



In [ ]:
pivot_df

In [ ]:
community_totals = pivot_df.drop('News_Number', axis=1).sum()

sorted_communities = community_totals.sort_values(ascending=False)

sorted_columns = ['News_Number'] + sorted_communities.index.tolist()
sorted_df = pivot_df[sorted_columns]

In [ ]:
sorted_df

In [ ]:
pd.merge(alt_number_5, sorted_df, how = 'left').drop(columns = 'News_Number')

In [ ]:
df_acc_alt
df_acc_wild

In [ ]:
df_acc_wild = df_cluster[df_cluster.Cluster_ID.isin(df_cluster5.Cluster_ID.tolist())][['Cluster_ID', 'User_ID']]



merged_df2 = pd.merge(df_acc_wild, df_partition, on='User_ID')

pivot_df2 = merged_df2.pivot_table(index='Cluster_ID', columns='Community_ID', aggfunc='size', fill_value=0)

pivot_df2.reset_index(inplace=True)

pivot_df2.columns.name = None  # Remove the 'Community_ID' label
pivot_df2.columns = [f"{col}" if col != 'Cluster_ID' else col for col in pivot_df2.columns]



In [ ]:
community_totals2 = pivot_df2.drop('Cluster_ID', axis=1).sum()

sorted_communities2 = community_totals2.sort_values(ascending=False)

sorted_columns2 = ['Cluster_ID'] + sorted_communities2.index.tolist()
sorted_df2 = pivot_df2[sorted_columns2]

In [ ]:
sorted_df2

In [ ]:
pd.merge(df_cluster5, sorted_df2, how = 'left')

In [ ]:
df_alt_camp_new = pd.merge(alt_number_5, sorted_df, how = 'left').drop(columns = 'News_Number')

In [ ]:
df_wild_camp_news = pd.merge(df_cluster5, sorted_df2, how = 'left').drop(columns = 'Cluster_ID')

In [ ]:
df_col_names = df_partition.groupby('Community_ID').count().reset_index().sort_values('User_ID', ascending = False).Community_ID

In [ ]:
df_col_names = df_col_names.tolist()

In [ ]:
df_col_names.insert(0, 'Campaign_ID')
df_col_names.insert(1, 'Campaign_Size')


In [ ]:
df_alt_camp_new[ df_col_names]

In [ ]:
pd.concat[df_alt_camp_new , df_wild_camp_news ]

In [ ]:
df_wild_camp_news['Community_49'] = 0

In [ ]:
df_wild_camp_news[df_col_names]

In [ ]:
df_new_final = pd.concat([df_alt_camp_new[ df_col_names], df_wild_camp_news[df_col_names]]).sort_values('Campaign_Size', ascending = False).iloc[1:].reset_index(drop = True)

In [ ]:
pd.DataFrame(df_partition.groupby('Community_ID').count().reset_index().sort_values('User_ID', ascending = False).User_ID.tolist())

In [ ]:
df_new_final.iloc[0][2:]

In [ ]:
import pandas as pd

df = pd.DataFrame({'A': [1, 2, 3],
                   'B': [4, 5, 6]})


df.loc[-1] = None
df = df.sort_index().reset_index(drop=True)

print(df)

In [ ]:
df_new_final.loc[-1] = None

In [ ]:
df_new_final = df_new_final.sort_index().reset_index(drop=True)

In [ ]:
df_new_final.iloc[0].Campaing_ID = 'Community_Info'

In [ ]:
df_new_final.iloc[0, df_new_final.columns.get_loc('Campaign_ID')] =  'Community_Size'
df_new_final.iloc[0, df_new_final.columns.get_loc('Campaign_Size')] =  1838


In [ ]:
df_new_final.iloc[0,2:] = df_partition.groupby('Community_ID').count().reset_index().sort_values('User_ID', ascending = False).User_ID.tolist()

In [ ]:
df_new_final.to_csv('Campaign_Table2.csv', index = False)

In [ ]:
df_new_final.loc[df_new_final.Community_0 > 0].sort_values('Community_0', ascending = False)

In [ ]:
df_acc_alt
df_acc_wild

In [ ]:
df_acc_alt['Campaign_ID'] = 'Alt_' + df_acc_alt['News_Number'].astype('str') 
df_acc_wild['Campaign_ID'] = 'Wild_' + df_acc_wild['Cluster_ID'].astype('str')

In [ ]:
all_acc_camp = pd.concat([df_acc_wild[['Campaign_ID', 'User_ID']],df_acc_alt[['Campaign_ID', 'User_ID']]])

In [ ]:
largest_community = max(partition.values(), key=lambda x: list(partition.values()).count(x))

largest_community_nodes = [node for node, comm_id in partition.items() if comm_id == largest_community]

largest_community_subgraph = G.subgraph(largest_community_nodes)


In [ ]:
partition_sub = community.best_partition(largest_community_subgraph)

In [ ]:
community.modularity(partition_sub, largest_community_subgraph)

In [ ]:
sub_communities_count = {}

for community_id in set(partition.values()):
    community_nodes = [node for node, comm_id in partition.items() if comm_id == community_id]
    
    community_subgraph = G.subgraph(community_nodes)
    
    sub_partition = community.best_partition(community_subgraph)
    
    num_sub_communities = len(set(sub_partition.values()))
    
    sub_communities_count[community_id] = num_sub_communities

print(sub_communities_count)

In [ ]:
df_sub_communities_count = pd.DataFrame.from_dict(sub_communities_count, orient='index', columns=['Num_Sub_Communities'])

df_sub_communities_count.reset_index(inplace=True)
df_sub_communities_count.rename(columns={'index': 'Community_ID'}, inplace=True)

In [ ]:
df_sub_communities_count['Community_ID'] = 'Community_'  + df_sub_communities_count['Community_ID'].astype('str')

In [ ]:
df_partition_number = df_partition.groupby('Community_ID').count().reset_index().sort_values('User_ID', ascending = False)

In [ ]:
pd.merge(df_partition_number , df_sub_communities_count)

In [ ]:
data = []

for community_id in set(partition.values()):
    community_nodes = [node for node, comm_id in partition.items() if comm_id == community_id]
    
    community_subgraph = G.subgraph(community_nodes)
    
    community_density = nx.density(community_subgraph)
    
    sub_partition = community.best_partition(community_subgraph)
    
    for user_id, sub_community_id in sub_partition.items():
        sub_community_label = f"{community_id}_{sub_community_id}"
        
        sub_community_nodes = [node for node, comm_id in sub_partition.items() if comm_id == sub_community_id]
        sub_community_subgraph = community_subgraph.subgraph(sub_community_nodes)
        
        sub_community_density = nx.density(sub_community_subgraph)
        
        data.append((user_id, community_id, sub_community_label, community_density, sub_community_density))

df_user_sub_community = pd.DataFrame(data, columns=['User_ID', 'Community_ID', 'Sub_Community_ID', 'Community_Density', 'Sub_Community_Density'])


In [ ]:
sub_com_total = df_user_sub_community.groupby('Sub_Community_ID').count().reset_index()[['Sub_Community_ID', 'User_ID']].sort_values('User_ID', ascending = False)

In [ ]:
sub_com_total 

In [ ]:
sub_com_total['Sub_Community_ID'] = 'Community_ID' + sub_com_total['Sub_Community_ID']

In [ ]:
sub_com_total = sub_com_total.reset_index(drop = False)

In [ ]:
grouped_data = df_user_sub_community.groupby('Community_ID').agg({'Community_Density': 'first', 'Sub_Community_Density': 'mean'}).reset_index()

grouped_data.columns = ['Community_ID', 'Community_Density', 'Sub_Community_Density']



In [ ]:
grouped_data

In [ ]:
sub_community_count = df_user_sub_community.groupby('Community_ID')['Sub_Community_ID'].nunique().reset_index()

sub_community_count.columns = ['Community_ID', 'Num_Sub_Communities']

sub_community_count

In [ ]:
df_user_sub_community.groupby('Community_ID').count().reset_index().sort_values('User_ID', ascending = False).reset_index(drop = True)[['Community_ID', 'User_ID']]

In [ ]:
df_user_sub_community.groupby('Sub_Community_ID').count()

In [ ]:
df_com_info_new = pd.merge(pd.merge(sub_community_count, grouped_data), df_user_sub_community.groupby('Community_ID').count().reset_index().sort_values('User_ID', ascending = False).reset_index(drop = True)[['Community_ID', 'User_ID']])

In [ ]:
df_com_info_new = df_com_info_new.sort_values('User_ID', ascending = False).reset_index(drop = True).rename(columns = {'User_ID': 'Community_Size'})

In [ ]:
df_com_info_new = df_com_info_new[['Community_ID', 'Community_Size', 'Num_Sub_Communities', 'Community_Density',
       'Sub_Community_Density'] ]

In [ ]:
df_user_sub_community

In [ ]:
pd.merge(df_com_info_new, community_sizes)

In [ ]:
df_com_info_new2 = df_com_info_new

In [ ]:
df_com_info_new2['Community_ID'] = 'Community_' + df_com_info_new2['Community_ID'].astype('str')

In [ ]:
df_com_info_new2=df_com_info_new2.rename(columns = {'Num_Sub_Communities': 'SubCommunity_Size', 'Sub_Community_Density': 'Average_SubCommunity_Density'})

In [ ]:
df_com_info_new2

In [ ]:
import pandas as pd


sub_community_sizes = df_com_info_new2.groupby('Community_ID')['SubCommunity_Size'].apply(list).reset_index()

df_com_info_new3 = pd.merge(df_com_info_new2, sub_community_sizes, on='Community_ID', how='left')



In [ ]:
df_com_info_new3

In [ ]:
df_sub_com_size = df_user_sub_community.groupby(['Community_ID', 'Sub_Community_ID']).count().reset_index()[['Community_ID', 'User_ID']].sort_values('User_ID', ascending = False)

In [ ]:
community_sizes = df_sub_com_size.groupby('Community_ID')['User_ID'].apply(list).reset_index()

print(community_sizes)

In [ ]:
community_sizes['Community_ID'] = 'Community_' + community_sizes['Community_ID'].astype('str')

In [ ]:
df_new_now = pd.merge(df_com_info_new2, community_sizes)

In [ ]:
df_com_info_new2

In [ ]:
df_new_now = df_new_now.rename(columns = {'SubCommunity_Size' : 'Total_SubCommunities', 'User_ID' : 'SubCommunity_Size'}) 

In [ ]:
df_new_now.columns

In [ ]:
df_new_now[['Community_ID', 'Community_Size', 'Total_SubCommunities', 'SubCommunity_Size',
       'Community_Density', 'Average_SubCommunity_Density',
       ]].to_csv('Sub_communities_Density.csv', index = False)

In [ ]:
df_user_sub_community_dict = df_user_sub_community[['User_ID', 'Sub_Community_ID']]

In [ ]:
df_user_sub_community_dict

In [ ]:
unique_sub_community_ids = df_user_sub_community_dict['Sub_Community_ID'].unique()

sub_community_id_to_int = {sub_id: i for i, sub_id in enumerate(unique_sub_community_ids)}

df_user_sub_community_dict['Sub_Community_Integer'] = df_user_sub_community_dict['Sub_Community_ID'].map(sub_community_id_to_int)


In [ ]:
df_user_sub_community_dict.columns

In [ ]:
sub_partition = df_user_sub_community_dict.set_index('User_ID')['Sub_Community_Integer'].to_dict()

In [ ]:
len(partition)

In [ ]:
len(sub_partition)

In [ ]:
print(community.modularity(partition, G))
print(community.modularity(sub_partition, G))

In [ ]:
df_user_sub_community_dict.columns

In [ ]:
df_user_sub_community_dict

In [ ]:
df_user_sub_community.loc[df_user_sub_community.Community_ID == 0].drop_duplicates('Sub_Community_ID')[['Sub_Community_ID', 'Sub_Community_Density']]

In [ ]:
df_user_sub_community.loc[df_user_sub_community.Community_ID == 0].groupby('Sub_Community_ID').count().reset_index(drop = False).sort_values('User_ID')

In [ ]:
df_new_now

In [ ]:
com_0_list = ['Community_0', 696, 0.03238,
'SubCommunity_0_0', 191, 0.078920,
'SubCommunity_0_4', 183, 0.064733,
'SubCommunity_0_2', 122, 0.073025,
'SubCommunity_0_6',119, 0.101268,
'SubCommunity_0_3', 75, 0.100541,
'SubCommunity_0_1', 3, 1.000000,
'SubCommunity_0_5', 3, 1.000000]

In [ ]:
df_com_0 = [com_0_list[i:i+3] for i in range(0, len(com_0_list), 3)]

df_com_0_result = pd.DataFrame(df_com_0, columns=['Community_ID', 'Community_Size', 'Community_Density'])

print(df_com_0_result)

In [ ]:
df_com_0_result.to_csv('Community_0.csv', index = False)

In [ ]:
df_new_now

In [ ]:
df_user_sub_community

In [ ]:



def calculate_clustering_coefficient(subgraph):
    return nx.average_clustering(subgraph)

results = []

for community_id in df_user_sub_community['Community_ID'].unique():
    community_df = df_user_sub_community[df_user_sub_community['Community_ID'] == community_id]
    print(len(community_df))
    G_community = nx.Graph()
    G_community.add_edges_from(community_df[['User_ID', 'User_ID']].values)
    clustering_coefficient = calculate_clustering_coefficient(G_community)
    results.append({'ID': f'Community_{community_id}', 'Clustering_Coefficient': clustering_coefficient})

for subcommunity_id in df_user_sub_community['Sub_Community_ID'].unique():
    subcommunity_df = df_user_sub_community[df_user_sub_community['Sub_Community_ID'] == subcommunity_id]
    G_subcommunity = nx.Graph()
    G_subcommunity.add_edges_from(subcommunity_df[['User_ID', 'User_ID']].values)
    clustering_coefficient = calculate_clustering_coefficient(G_subcommunity)
    results.append({'ID': f'SubCommunity_{subcommunity_id}', 'Clustering_Coefficient': clustering_coefficient})

df_results_co = pd.DataFrame(results)

print(df_results_co)


In [ ]:
 df_user_sub_community['Community_ID'].unique()

In [ ]:
all_acc_camp

In [ ]:
community_0_analysis = pd.merge(all_acc_camp, df_user_sub_community.loc[df_user_sub_community.Community_ID == 0])

In [ ]:
community_0_analysis['Community_ID'] = 'Community_' + community_0_analysis['Community_ID'].astype('str')
community_0_analysis['Sub_Community_ID'] = 'SubCommunity_' + community_0_analysis['Sub_Community_ID'].astype('str')

In [ ]:
df_new_final[['Campaign_ID', 'Campaign_Size', 'Community_0']]

In [ ]:
community_0_analysis.columns

In [ ]:
campaign_subcommunity_counts = community_0_analysis.groupby(['Campaign_ID', 'Sub_Community_ID'])['User_ID'].nunique().reset_index()
campaign_subcommunity_counts.columns = ['Campaign_ID', 'Sub_Community_ID', 'Accounts_From_Subcommunity']

pivot_df2 = campaign_subcommunity_counts.pivot(index='Campaign_ID', columns='Sub_Community_ID', values='Accounts_From_Subcommunity').fillna(0)

pivot_df2 = pivot_df2.reindex(sorted(pivot_df2.columns), axis=1)


In [ ]:
pivot_df2 = pivot_df2.reset_index(drop = False)

In [ ]:
pivot_df2 

In [ ]:
pivot_df2[['Campaign_ID', 'SubCommunity_0_0', 'SubCommunity_0_1',
       'SubCommunity_0_2', 'SubCommunity_0_3', 'SubCommunity_0_4',
       'SubCommunity_0_5', 'SubCommunity_0_6']]

In [ ]:
df_com0_analysis = pd.merge(df_new_final[['Campaign_ID', 'Campaign_Size', 'Community_0']], pivot_df2)

In [ ]:
SubCommunity_0_0	191	0.07892
SubCommunity_0_4	183	0.064733
SubCommunity_0_2	122	0.073025
SubCommunity_0_6	119	0.101268
SubCommunity_0_3	75	0.100541
SubCommunity_0_1	3	1
SubCommunity_0_5	3	1

In [ ]:
df_com0_analysis.columns

In [ ]:
df_com0_analysis = df_com0_analysis[['Campaign_ID', 'Campaign_Size', 'Community_0','SubCommunity_0_0',
                 'SubCommunity_0_4', 'SubCommunity_0_2', 'SubCommunity_0_6',
                 'SubCommunity_0_3', 'SubCommunity_0_1', 'SubCommunity_0_5']]

In [ ]:
df_com0_analysis = df_com0_analysis.sort_values(['Community_0', 'Campaign_Size'], ascending = False)

In [ ]:
df_com0_analysis['Community_0_Percentage'] = (df_com0_analysis.Community_0 /df_com0_analysis.Campaign_Size) * 100

In [ ]:
df_com0_analysis.sort_values('Community_0_Percentage', ascending = False)

In [ ]:
df_com0_analysis =df_com0_analysis.reset_index(drop = True)

In [ ]:
df_com0_analysis.head(5)

In [ ]:
df_com0_analysis.columns

In [ ]:
import pandas as pd


new_row = ['Community_0_Size',-1, 696, 191,183,122,119,75,3,3, -1]  # Example new row

new_row_df = pd.DataFrame([new_row], columns=df_com0_analysis.columns)

df_com0_analysis = pd.concat([new_row_df, df_com0_analysis], ignore_index=True)


In [ ]:
df_com0_analysis = df_com0_analysis[['Campaign_ID', 'Campaign_Size', 'Community_0','Community_0_Percentage',
                  'SubCommunity_0_0','SubCommunity_0_4', 'SubCommunity_0_2', 'SubCommunity_0_6',
                  'SubCommunity_0_3', 'SubCommunity_0_1', 'SubCommunity_0_5',
       ]]

In [ ]:
df_com0_analysis.to_csv('Community_0_Campaign_Table2.csv', index = False)

In [ ]:
df_com0_analysis.loc[(df_com0_analysis.Campaign_Size > 10) & (df_com0_analysis.Community_0_Percentage >= 50)]

In [ ]:
df_com0_analysis.columns

In [ ]:
mask = ~(df_com0_analysis['Community_0'] == df_com0_analysis[['SubCommunity_0_0', 'SubCommunity_0_4', 
                                                              'SubCommunity_0_2','SubCommunity_0_6', 
                                                              'SubCommunity_0_3', 'SubCommunity_0_1',
                                                              'SubCommunity_0_5']].sum(axis=1))

result = df_com0_analysis[mask]

In [ ]:
result.Campaign_ID.tolist()

In [ ]:
result

In [ ]:
df_com0_analysis

In [ ]:
df_com0_analysis.loc[df_com0_analysis.Campaign_ID  == 'Alt_137']

In [ ]:
all_acc_camp.loc[all_acc_camp.Campaign_ID == 'Alt_137']

In [ ]:
df_user_sub_community.loc[ df_user_sub_community.Community_ID == 0]

In [ ]:
pd.merge(all_acc_camp.loc[all_acc_camp.Campaign_ID == 'Alt_209'], df_user_sub_community.loc[ df_user_sub_community.Community_ID == 0]).groupby('Sub_Community_ID').count().reset_index()

In [ ]:
all_acc_camp

In [ ]:
for i in result.Campaign_ID.tolist():
    temp = pd.merge(all_acc_camp.loc[all_acc_camp.Campaign_ID == i], df_user_sub_community.loc[ df_user_sub_community.Community_ID == 0]).groupby('Sub_Community_ID').count().reset_index()[['Sub_Community_ID', 'User_ID']]
    temp['Sub_Community_ID'] = 'Sub_Community_' + temp['Sub_Community_ID'].astype('str')
    for index,row in temp.iterrows():
        temp2 = df_com0_analysis.loc[df_com0_analysis.Campaign_ID == i, row.Sub_Community_ID] = row.User_ID

In [ ]:
df_com0_analysis.columns

In [ ]:
df_com0_analysis = df_com0_analysis.drop(columns = ['Sub_Community_0_0', 'Sub_Community_0_2',
       'Sub_Community_0_3', 'Sub_Community_0_4', 'Sub_Community_0_6',
       'Sub_Community_0_1'], axis = 0)

In [ ]:
df_user_sub_community.loc[df_user_sub_community.Community_ID == 6]

In [ ]:
raja_list_community_6 = pd.merge(df_user_sub_community.loc[df_user_sub_community.Community_ID == 6],df_account)[['User_ID', 'UserDescription', 
                                                                                         'ScreenName', 'UserFollowers', 
                                                                                         'UserFriends', 'TotalStatus', 
                                                                                         'Political', 'party']]

In [ ]:
raja_list_community_6['UserLink'] = 'https://twitter.com/i/user/' + raja_list_community_6.User_ID.astype('str')

In [ ]:
raja_list_community_6.sort_values('UserFollowers', ascending = False).to_csv('Community_6_Manual_Check.csv', index = False)

In [ ]:
raja_list_community_6.groupby('party').count()

In [ ]:
raja_list_community_6

In [ ]:
df_alt_influence

In [ ]:
df_alt_topic = pd.read_csv('AltNews_Topics.csv')

In [ ]:
df_alt_topic['Campaign_ID'] = 'Alt_' + df_alt_topic['News_Number'].astype('str')

In [ ]:
filtered_df = df_new_final.loc[df_new_final['Campaign_ID'].str.contains('Alt')]

In [ ]:
filtered_df6 = filtered_df.loc[filtered_df.Community_6 > 1][['Campaign_ID', 'Campaign_Size', 'Community_6']].sort_values('Community_6', ascending = False)

In [ ]:
filtered_df6 = pd.merge(df_alt_topic, filtered_df)[['Campaign_ID', 'Community_6', 'Topic_Number' ]].dropna()

In [ ]:
filtered_df6.loc[filtered_df6.Community_6 >= 5]